In [4]:
import pandas as pd
import os 
from loguru import logger 



In [3]:
CSV_FILE_PATH = "/quotes_db.csv"

In [ ]:
def write_db(df: pd.DataFrame):
    df.to_csv(CSV_FILE_PATH, index=True, index_label='id')

def read_db()->pd.DataFrame:
    df = pd.read_csv(CSV_FILE_PATH, index_cols='id')
    return df

def initialize_db():
    df = pd.DataFrame(columns=['id', 'text'])
    df = df.set_index('id')
    write_db(df)


initialize_db()

In [1]:
"toto" + "tata"

'tototata'

In [11]:
# backend/modules/df_tools.py
from loguru import logger 
import pandas as pd
import os 
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base, sessionmaker, session


DB_FILE_PATH = os.path.join(os.getcwd(),"quotes_db.sql")
engine = create_engine(f"sqlite:///{DB_FILE_PATH}", echo=False)
session_locale = sessionmaker(bind=engine, autoflush=False, autocommit=False)

Base = declarative_base()



#CSV_FILE_PATH = os.path.join("backend","data", "quotes_db.csv")




In [12]:
class Quote(Base):
    __tablename__ = "quotes"
    id = Column(Integer, primary_key=True)
    text = Column(String, nullable=False)

TABLE_NAME = "quotes"

In [13]:
def get_session():
    return session_locale() # permet de créer une session quand on en a besoin

def write_db(df: pd.DataFrame):
    #df.to_csv(CSV_FILE_PATH, index=True, index_label='id')
    #print("write_df\n", df)
    df.to_sql(
        TABLE_NAME,
        con=engine,
        if_exists='replace', # supprime et remplace la db si deja existante avec la nouvelle db modifiée
        index=True,
        index_label='id'
    )

def read_db()->pd.DataFrame:
    # df = pd.read_csv(CSV_FILE_PATH, index_col='id')

    # if df.isnull().any().any():
    #     # raise ValueError("Le CSV contient des valeurs nulles")
    #     df_clean = df.dropna()
    # for r, c in df.iterrows():
    #     print(c.text.isempty)
    with get_session() as session:
        quotes = session.query(Quote).all() # la liste des objets Quote

    # on veut transformer en dataframe comme dans df_tools 
    # on parcout la liste quotes
    data = [{"id": q.id, "text": q.text} for q in quotes]
    
    if not data:
        # cas index vide pris en compte
        return pd.DataFrame(columns=["text"])
    
    df = pd.DataFrame(data).set_index("id")

    # nettoayge NAN comme dans 
    for index, row in df.iterrows() : 
        for col in df.columns:
            if pd.isna(row[col]):
                logger.info(f"NaN trouvé à la ligne {index}, colonne '{col}' remplacé par la valeur 'NULL'")
                df.loc[index, col] = "NULL_REPLACEMENT_VALUE"
    
    # print("DATA =", data)
    # print("COLUMNS =", df.columns)

    return df

def initialize_db():
    # if os.path.exists(CSV_FILE_PATH):
    #     logger.info("La base de données existe")
    # else:
    #     logger.info(f"impossible de trouver le fichier {CSV_FILE_PATH}")
    #     df = pd.DataFrame(columns=['id', 'text'])
    #     df = df.set_index('id')
    #     write_db(df)
    #     logger.info(f"le fichier {CSV_FILE_PATH} a été créé")

    if os.path.exists(DB_FILE_PATH):
        logger.info("La base de données existe déjà")
    else:
        Base.metadata.create_all(bind=engine) 
        logger.info(f"Base de données SQL {DB_FILE_PATH} créée")

In [14]:
initialize_db()

2025-12-09 10:50:07.793 | INFO     | __main__:initialize_db:62 - Base de données SQL c:\Users\TIM\Documents\Info\Simplon\PythonProjet\FASTAPI\FAST_API_INITIATION\DEV\quotes_db.sql créée


In [17]:
print(read_db())

               text
id                 
1   citation_test_1
2   citation_test_2


In [16]:
df_test = pd.DataFrame(
    {"text": ["citation_test_1", "citation_test_2"]},
    index=[1,2]
)
write_db(df_test)